# Advent of code 2022

In [84]:
import re

import pandas as pd

## Part 1

In [85]:
test_input='''    [D]    
[N] [C]    
[Z] [M] [P]
 1   2   3 

move 1 from 2 to 1
move 3 from 1 to 3
move 2 from 2 to 1
move 1 from 1 to 2'''

In [86]:
with open('data/day05.txt') as fIn:
    puzzle_input=fIn.read()

Let's start by splitting the input into the initial state and the instructions:

In [87]:
state=test_input.split('\n\n')[0]
instructions=test_input.split('\n\n')[1]

Rather than mess about with a parser, let's just stick the input into a DataFrame, so I can easily restructure it:

In [88]:
df=pd.DataFrame(reversed([list(row) for row in state.splitlines()])).T
df

,0,1,2,3
0,,[,[,
1,1,Z,N,
2,,],],
3,,,,
4,,[,[,[
5,2,M,C,D
6,,],],]
7,,,,
8,,[,,
9,3,P,,


And get down to just the rows which represent piles:

In [89]:
df=df[df[0]!=' '].set_index(0)
df

,1,2,3
0,,,
1,Z,N,
2,M,C,D
3,P,,


Now it'll be easiest to map that back onto a dictionary of lists:

In [90]:
# reverse the list so we're taking off the "front"

initial_state={i:[l for l in reversed(list(ls)) if l!=' ']
               for (i, ls) in df.iterrows()}

initial_state

{'1': ['N', 'Z'], '2': ['D', 'C', 'M'], '3': ['P']}

Good. Now let's do the instructions:

In [91]:
s=initial_state.copy()
print(s)
for (num, src, dst) in re.findall('move (\d+) from (\d+) to (\d+)', instructions):
    n=int(num)
    
    # Crates moved one at a time, so reverse
    # Python annoyingly doesn't just reverse the list, but
    # gives an iterator which doesn't recognise '+'
    s[dst]=list(reversed(s[src][:n])) + s[dst]
    s[src]=s[src][n:]
    print(s)

{'1': ['N', 'Z'], '2': ['D', 'C', 'M'], '3': ['P']}
{'1': ['D', 'N', 'Z'], '2': ['C', 'M'], '3': ['P']}
{'1': [], '2': ['C', 'M'], '3': ['Z', 'N', 'D', 'P']}
{'1': ['M', 'C'], '2': [], '3': ['Z', 'N', 'D', 'P']}
{'1': ['C'], '2': ['M'], '3': ['Z', 'N', 'D', 'P']}


OK, that seems to work properly. So stick it all in a function:

In [92]:
def day05_a(str_in):
    
    # sort out the state first
    
    state_str=str_in.split('\n\n')[0]
    
    state_df=pd.DataFrame(reversed([list(row) for row in state_str.splitlines()])).T
    
    state_df=state_df[state_df[0]!=' '].set_index(0)
    
    state={i:[l for l in reversed(list(ls)) if l!=' ']
                   for (i, ls) in state_df.iterrows()}
    
    # Then apply the instructions:
    
    instructions_str=str_in.split('\n\n')[1]
    
    for (num, src, dst) in re.findall('move (\d+) from (\d+) to (\d+)', instructions_str):
        n=int(num)
    
        # Crates moved one at a time, so reverse
        # Python annoyingly doesn't just reverse the list, but
        # gives an iterator which doesn't recognise '+'
        state[dst]=list(reversed(state[src][:n])) + state[dst]
        state[src]=state[src][n:]
    
    # And finally, get the top crate on each stack:
    
    out=''.join([c[1] for c in sorted([(i, l[0]) for (i, l) in state.items()])])
    
    return out

In [93]:
assert day05_a(test_input)=='CMZ'

In [94]:
day05_a(puzzle_input)

'JCMHLVGMG'

## Part 2

OK, this is going back to how I'd originally interpreted the question. So just need to remove the `reversed` function in the main loop.

In [97]:
def day05_b(str_in):
    
    # sort out the state first
    
    state_str=str_in.split('\n\n')[0]
    
    state_df=pd.DataFrame(reversed([list(row) for row in state_str.splitlines()])).T
    
    state_df=state_df[state_df[0]!=' '].set_index(0)
    
    state={i:[l for l in reversed(list(ls)) if l!=' ']
                   for (i, ls) in state_df.iterrows()}
    
    # Then apply the instructions:
    
    instructions_str=str_in.split('\n\n')[1]
    
    for (num, src, dst) in re.findall('move (\d+) from (\d+) to (\d+)', instructions_str):
        n=int(num)
    
        # Crates moved together, so don't reverse :-)
        state[dst]=state[src][:n] + state[dst]
        state[src]=state[src][n:]
    
    # And finally, get the top crate on each stack:
    
    out=''.join([c[1] for c in sorted([(i, l[0]) for (i, l) in state.items()])])
    
    return out

In [99]:
assert day05_b(test_input)=='MCD'

In [100]:
day05_b(puzzle_input)

'LVMRWSSPZ'